<img src="https://lh3.googleusercontent.com/pw/ACtC-3fFHZrzKpHGWl0vYz7Sr8FX8QqLQ_tc8XHBSwqQnM4hgsIOjtjaOde1M9oHSAfe1Fs2SwVORlapit4-JOz0mjP8Tnz6HetkLZDZb8CifSd0uoSp1Nj3wG_wh1sEQlKXXzvEA9Y9HnQqu2Ecv2igmInb=w1097-h235-no?authuser=0" alt="2020年度ゲノム情報解析入門" height="100px" align="middle">

# 機械学習 - 勾配法 -

　前回は、scikit-learnの`LinearRegression()`で線形回帰モデルを構築しました。線形回帰モデルの学習では、残差の二乗の合計値、**目的関数**である**残差平方和 (residual sum of squares)** が最も小さくなる係数 $\beta$ と誤差 $e$ の直線を求めていました。

$$ 残差平方和: \sum_{i=1}^{N} (\hat{y}_{i} - y_i)^2 $$

その際、**最小二乗法**と呼ばれる「解析的」な方法でパラメータ（係数 $\beta$ や誤差 $e$ ）の最適値を求めていました。

<small>※ 「解析的」とは、連立方程式や微分等の数式の変形により、厳密解を求めること。</small>

　しかし、選んだモデルや手法、目的関数によっては簡単に解の求まる連立方程式などの数式に落とし込めない場合があります。また、数式に落とし込めても計算時間がかかりすぎてしまうこともあります。そうした場合には、「数値的」な方法でパラメータ推定をおこなうことが機械学習では良くあります。

<small>※ 「数値的」とは、コンピュータを使って近似解を求めること。</small>

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/bibun.png?raw=true" alt="gradient_method" height="300px">

　今回、**勾配法（Gradient method）**と呼ばれる数値的にパラメータ推定をおこなう方法を学びます。


### 最小二乗法の計算コスト
<small>※ 数学（行列）を使ったやや込み入った説明をしています。読み飛ばしてOKです。</small>

　$i$行$j$列のトレーニングデータセットの説明変数を行列$X$で表すと、次のようになります。1行は1サンプル分のデータで、各列は説明変数の値です。また、目的変数$y$も$i$行1列の行列で表すことができ、係数$\beta$も$j$行1列の行列で表すことができます。

$$
X = \begin{pmatrix}
x_{11} & x_{12} & x_{12} & ... & x_{1j} \\
x_{21} & x_{22} & x_{22} & ... & x_{2j} \\
x_{31} & x_{32} & x_{32} & ... & x_{3j} \\
 : & : & : & ... & : \\
 x_{i1} & x_{i2} & x_{i2} & ... & x_{ij} \\
\end{pmatrix}, 
y = \begin{pmatrix}
y_{1}  \\
y_{2} \\
y_{3} \\
 :  \\
y_{i} \\
\end{pmatrix},  
\beta = \begin{pmatrix}
\beta_{1}  \\
\beta_{2} \\
\beta_{3} \\
 :  \\
\beta_{j} \\
\end{pmatrix}
$$

　最小二乗法で、残差平方和を最小化する$\hat{\beta}$を求めるとき、実際には次のような行列式を解いています。
$$ \hat{\beta} = (X^{T} X)^{-1} X^{T} y $$

　説明変数$j$が多くなると、上述の式の$(X^{T} X)^{-1}$（逆行列）の計算が大規模計算機を用いても困難になってきたり、そもそも求まらなかったりします（時間がかかりすぎるか、メモリ上に数値を保持できなくなります）。

　もう少し具体的に言うと、
- $X^{T}$（$j\times i$行列）と$X$（$i\times j$行列）の積$X^{T} X$により、$j\times j$行列ができます。この$j\times j$行列の逆行列を求めることになります。
- LU分解を利用して $j\times j$行列の逆行列を得ようとすると、$j^3$の計算量が必要になります（「[プログラミングのための線形代数](https://www.ohmsha.co.jp/book/9784274065781/)」より）。


### 勾配法とは？

　線形回帰モデルは次のような式で表されます。
$$y=\beta_1 x_1 + \beta_2 x_2 + ... + \beta_m x_m + e $$

　最小二乗法と勾配法では、パラメータの求め方がどう違うのか？

　最小二乗法は、計算により最小値をピンポイントに見つける方法です。一方で、勾配法は、ある値から出発して、係数$\beta$や誤差$e$の値を徐々に更新し、最小値に近づいていく方法です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/gradient_method.png?raw=true" alt="gradient_method" height="250px">

### 勾配法のアルゴリズム

　勾配法は、上述したとおり、**ある値から出発して、係数𝛽や誤差𝑒の値を徐々に更新し、最小値に近づいていく方法** です。その方法について、もう少し具体的な話をしましょう。

　次の図のように、下に凸なグラフがあり、現在の$\beta$の値が$\beta_0$とします。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/gradient_method_example.png?raw=true" alt="gradient_method_example" height="250px">

　目的関数$Cost(\beta)$が最小値となる$\beta_{optimum}$に近づくためには、現在の$\beta$からどのように値を変化させればいいでしょうか？  
> $\beta$が右側に移るような値を$\beta_0$に追加する

どの程度の値を追加すれば良いでしょうか？
> あまり大きすぎない値を追加する。

　勾配法では、次のような処理をおこない、パラメータの最適値に近づきます。
1. まず、$\beta_0$の「接線の傾き」を調べます。すなわち、$Cost(\beta)$を$\beta$について微分したときの値を調べます。
  $$ \frac{\partial Cost(\beta)}{\partial  \beta} $$

1. その値が負のとき、右方向に移るように$\beta$の値を更新します。その時の更新幅を、傾きの大きさと**学習率$\eta$（イータ; eta）**の積で決定します。
  $$ \beta_{1} := \beta_{0} - \eta \frac{\partial Cost(\beta)}{\partial  \beta} $$

1. この1と2のステップを$\beta$の値が変化しなくなるまで（もしくは、指定の繰り返し数に達するまで）、ひたすら繰り返します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/gradient_method_algorithm.png?raw=true" alt="gradient_method_algorithm" height="250px">


### 実習で使用するデータセット

　次のコードセルを実行して、データファイル（[gene_expression.csv](https://github.com/CropEvol/lecture/blob/master/textbook_2019/dataset/gene_expression.csv)）をダウンロードしてください。

ファイルの詳細:
- ファイル名: gene_expression.csv
- カンマ区切りテキストファイル
- 100行（100サンプル） x 51列（表現型値 + 50個の遺伝子発現量）

今回の実習では、一部の列データのみ使います。
- 表現型値 `phenotype`
- 遺伝子11の発現量 `gene_11` (例題)
- 遺伝子7, 11, 29の発現量 `gene_7`,`gene_11`,`gene_29`  (実習問題)


In [ ]:
# /// 実習前にこのセルを実行してください ///
# データの読み込み
!wget -q -O gene_expression.csv https://raw.githubusercontent.com/CropEvol/lecture/master/data/gene_expression.csv
# 動画再生用モジュール
!wget -q -O gradient_method.py https://raw.githubusercontent.com/CropEvol/lecture/master/modules/gradient_method.py
!apt-get -q install ffmpeg

# pandasで読み込み
import pandas as pd
df = pd.read_csv("gene_expression.csv", sep=",", header=0)
df

---

## 今回の実習内容

1. 前処理
1. 確率的勾配降下法 Stochastic Gradient Descent


### 1. 前処理

　前回と同じく、勾配法の仕組みを学ぶために、説明変数1個と目的変数1個のみを使います。

- 説明変数: 遺伝子11の発現量 `gene_11`（他の遺伝子でも構いません）
- 目的変数: 表現型値 `phenotype`

> gene_11の遺伝子発現量から表現型値（phenotype）を予測する線形回帰モデルを作る
  $$ y = \beta_{gene\_11} x_{gene\_11} + e $$

　次のコードセルを実行して、以下の前処理をおこないましょう。
1. 説明変数xと目的変数yの準備
2. 説明変数のスケーリング
3. データのグラフ描画

<small>※ 勾配法の仕組みを勉強するだけなので、「トレーニングデータとテストデータの分割」や「モデルの評価」、「新しいデータの予測」は今回おこないません。</small>





In [ ]:
# 説明変数xとして使うデータ
# gene_1からgene_50であれば、どれでもOK（初期値: gene_11）
use_col = "gene_11"  

#===== 以下は変更しないでください =====
# 変数x, y
import numpy as np
x = np.array(df[use_col]).reshape(-1,1)
y = np.array(df["phenotype"])

# 説明変数xの正規化
from sklearn.preprocessing import MinMaxScaler 
mms = MinMaxScaler()
x = mms.fit_transform(x)

# グラフ
import matplotlib.pyplot as plt
plt.scatter(x, y, color="blue", label="training data") # データ
plt.xlabel("gene expression (normalized)")  # x軸ラベル
plt.ylabel("phenotype value")               # y軸ラベル
plt.legend()  # 凡例
plt.show()

### 参考: 最小二乗法による線形回帰モデルを作った場合…

　scikit-learnの`LinearRegression()`（最小二乗法）で線形回帰モデルを構築すると、次のコードセルの実行結果に表示される係数$\beta$と誤差$e$が得られます。

In [ ]:
# =================== 線形回帰 ===================
from sklearn.linear_model import LinearRegression

# モデル作成・学習
model_lr = LinearRegression()
model_lr.fit(x, y)

# 学習後の係数と誤差を確認
print("b=", model_lr.coef_)
print("e=", model_lr.intercept_)

# 決定係数
print("R2=", model_lr.score(x, y))

# =================== グラフ ===================
import matplotlib.pyplot as plt

# 直線を書くためのデータ
x_line = np.linspace(np.min(x), np.max(x), num=2)
y_line = model_lr.predict(x_line.reshape(-1, 1))

# グラフ
import matplotlib.pyplot as plt
plt.scatter(x, y, color="blue", label="data") # データ
plt.plot(x_line, y_line, color="orange", label="predict") # 予測モデル
plt.xlabel("gene expression (normalized)")  # x軸ラベル
plt.ylabel("phenotype value")               # y軸ラベル
plt.legend()  # 凡例
plt.show()

### 2. 確率的勾配降下法 Stochastic Gradient Descent

　**確率的勾配降下法（Stochastic Gradient Descent: SGD）**は、ランダムに選んだ1サンプルを使って勾配計算をおこない、係数$\beta$や誤差$e$などの**パラメータ（Parameter）**を少しずつ更新する方法です。


1. パラメータ（係数や誤差）の初期値を設定する。通常、「0」または「ごく小さいランダムな数値（小数点数）」が初期値として設定される。
1. 以下(1)〜(3)の操作を、指定のトレーニング回数繰り返す。  
  (1) データセットからランダムに1サンプル選び、そのデータを使って、勾配計算をおこなう。  
  (2) 勾配と学習率をかけた値を、パラメータから引き算し、パラメータ更新をおこなう。
  $$ 更新後パラメータ := 更新前パラメータ - 学習率 \times 勾配$$
  (3) (1)と(2)を全サンプルに対しておこなう。


　「確率的勾配降下法」を使った線形回帰モデルの構築は、scikit-learnで簡単に実装できます。

```python
# 確率的勾配降下法による線形回帰モデル
from sklearn.linear_model import SGDRegressor
モデル変数 = SGDRegressor(max_iter=トレーニング回数, eta0=学習率)
モデル変数.fit(説明変数, 目的変数)
```

In [ ]:
# ===== 確率的勾配降下法 =====
# モデル作成・学習
from sklearn.linear_model import SGDRegressor
model_sgd = SGDRegressor(max_iter=100, eta0=0.01, learning_rate="constant")
model_sgd.fit(x, y)
print("b=", model_sgd.coef_)
print("e=", model_sgd.intercept_)

# 決定係数
r2 = model_sgd.score(x, y)
print("R2=", r2)

# グラフ
import matplotlib.pyplot as plt

x_line = np.linspace(np.min(x), np.max(x), num=2)
y_line = model_sgd.predict(x_line.reshape(-1, 1))

fig = plt.figure(figsize=[8, 4])
plt.scatter(x, y, color="blue") # データ
plt.plot(x_line, y_line, color="orange")
plt.xlabel("normalized x")  # x軸ラベル
plt.ylabel("y")  # y軸ラベル
plt.show()

### 実習1

　実習問題では、3つの説明変数を使って、次の線形回帰モデルを作ります。
> gene_7, 11, 28の遺伝子発現量から表現型値（phenotype）を予測する線形回帰モデルを作る
  $$ y = \beta_{gene\_7} x_{gene\_7} + \beta_{gene\_11} x_{gene\_11} + \beta_{gene\_28} x_{gene\_28} + e $$


　「確率的勾配降下法」で線形回帰モデルのパラメータ推定（係数 $\beta$ と 誤差 $e$の値の推定）をおこなってください。その際、トレーニング回数や学習率に適当な値を設定し、決定係数 $R^2>0.5$ を目指してください。

```python
# 確率的勾配降下法による線形回帰モデル
from sklearn.linear_model import SGDRegressor
モデル変数 = SGDRegressor(max_iter=トレーニング回数, eta0=学習率)
モデル変数.fit(説明変数, 目的変数)
```




In [ ]:
# 使用するデータの準備
import numpy as np
xx = np.array(df[["gene_7", "gene_11", "gene_28"]]) # 説明変数
yy = np.array(df["phenotype"]) # 目的変数

# 説明変数の正規化
from sklearn.preprocessing import MinMaxScaler 
mms = MinMaxScaler()
xx = mms.fit_transform(xx)

# --------------- 編集箇所: start ---------------
# 学習前のモデルを作成
from sklearn.linear_model import SGDRegressor
model_sgd2 = SGDRegressor(max_iter=1, eta0=1, learning_rate="constant")
# データをセットしてモデルを学習
model_sgd2
# --------------- 編集箇所: end -----------------

# 係数と誤差、決定係数
print("b=", model_sgd2.coef_)
print("e=", model_sgd2.intercept_)
print("R2=", model_sgd2.score(xx, yy))

#### 解答例

In [ ]:
# 変数x, y
import numpy as np
xx = np.array(df[["gene_7", "gene_11", "gene_28"]])
yy = np.array(df["phenotype"])

# 説明変数xの正規化
from sklearn.preprocessing import MinMaxScaler 
mms = MinMaxScaler()
xx = mms.fit_transform(xx)

# --------------- 編集箇所: start ---------------
# 学習前のモデルを作成
from sklearn.linear_model import SGDRegressor
model_sgd2 = SGDRegressor(max_iter=1000, eta0=0.05, learning_rate="constant")
# データをセットしてモデルを学習
model_sgd2.fit(xx, yy)
# --------------- 編集箇所: end -----------------

# 係数と誤差、決定係数
print("b=", model_sgd2.coef_)
print("e=", model_sgd2.intercept_)
print("R2=", model_sgd2.score(xx, yy))

### 学習過程の一部を観察する

　コードの詳しい解説はおこないませんが、次の2つのコードセルを実行すると、確率的勾配降下法による学習過程の一部を動画にすることができます。

<small>*※ 学習過程のデータを動画用データに変換するコードは、こちらに記述しています: [gradient_method.py](https://github.com/CropEvol/lecture/blob/master/textbook_2019/modules/gradient_method.py)の`plot_reg`関数*</small>

In [ ]:
# 説明変数xとして使うデータ
# gene_1からgene_50であれば、どれでもOK（初期値: gene_11）
use_col = "gene_11"

# 確率的勾配降下法の設定値
n_iter = 100  # 学習回数
eta   = 0.1   # 学習率 

# ==========　以下は変更しないでください ==========
# ========== 前処理 ==========
# 説明変数x、目的変数y
import numpy as np
x = np.array(df[use_col]).reshape(-1,1)
y = np.array(df["phenotype"])

# 説明変数xの正規化
from sklearn.preprocessing import MinMaxScaler 
mms = MinMaxScaler()
x = mms.fit_transform(x)

# ========== モデル作成・学習 ==========
from copy import deepcopy
# 記録用リスト
log_coef = []      # 係数（傾き）
log_intercept = [] # 誤差(切片)
log_cost = []      # 残差二乗和

# モデル作成
from sklearn.linear_model import SGDRegressor
model_gd = SGDRegressor(eta0=eta, random_state=1, learning_rate="constant")

# 設定数の学習を繰り返す
for iteration in range(n_iter):
  # トレーニングデータをランダムに並べ替える
  r = np.random.permutation(len(y))

  # 1サンプルずつ学習する
  for i in range(0, len(y), 1):
    idx = r[i:i+1]  # サンプルのインデックスを取得
    x_i = x[idx]       # サンプルのデータを取得
    y_i = y[idx]
    # 学習
    model_gd.partial_fit(x_i, y_i)
  
  # 記録
  log_coef.append(deepcopy(model_gd.coef_))
  log_intercept.append(model_gd.intercept_)
  cost = ((model_gd.predict(x) - y)**2).sum() / 2.0 / len(y)  # コスト（残差二乗和）の計算
  log_cost.append(cost)

# ========== 学習後のパラメータと決定係数 ==========
# 学習後の係数と誤差を確認
print("b=", model_gd.coef_)
print("e=", model_gd.intercept_)
# 決定係数
print("R2=", model_gd.score(x, y))

# ========== 動画 ==========
import matplotlib.pyplot as plt
from matplotlib import animation, rc, rcParams
from IPython.display import HTML
from gradient_method import plot_reg
rcParams['animation.embed_limit'] = 2**128

# パラメータのログを取得
b_ = log_coef
e_ = log_intercept
c_ = log_cost

# 動画実行
fig = plt.figure(figsize=[16, 4])
plt.close()
frames = plot_reg(fig, x, y, b_, e_, c_, n_frames=100)
ani = animation.ArtistAnimation(fig, frames, interval=100, blit=True)
# ani.save('gradient_descent.mp4', writer="ffmpeg")
rc('animation', html='jshtml')
ani

### おもな勾配法

　「確率的勾配降下法」の他にも勾配法はいくつかあります。代表的なものとして以下の3つが挙げられます。これらの違いは、勾配の計算をおこなうときに使うサンプルセットが異なっている点です。そのほかのアルゴリズムはほとんど同じです。

- **確率的勾配降下法 Stochastic Gradient Descent**
  - ランダムに選んだ1サンプルのデータを使って勾配計算をおこない、その値を使ってパラメータを更新する
  $$ 更新後パラメータ := 更新前パラメータ - 学習率 \times 勾配$$
- **最急降下法（バッチ勾配降下法） Gradient Descent**
  - データセットの全サンプルに対して勾配計算をおこない、その平均値でパラメータを更新する。  
  $$ 更新後パラメータ := 更新前パラメータ - 学習率 \times \frac{1}{N}\sum 勾配$$

- **ミニバッチ勾配降下法 Mini batch Gradient Descent**
  - 確率的勾配降下法と最急降下法の中間的な手法。
  - ランダムに選んだ複数サンプルのデータセットを使って勾配の計算をおこない、パラメータを更新する


　また、以下に、「確率的勾配降下法」と「最急降下法」のおもな違いを表にまとめています。

|| 確率的勾配降下法 | 最急降下法 |
|---:|---:|---:|
| パラメータ更新 | 1サンプル毎 | データセット毎 |
| 計算量 | 一定 | サンプル数に依存して多くなる |
| 解周辺に辿り着く早さ | 早い | サンプル数に依存して遅くなる |
| 1サンプル（外れ値）の影響 | 影響を受けやすい | 平均値を使うので影響が緩和される |
| パラメータ更新の方向<br>（学習率が低い場合） | 総じて目標の解の方向に更新されるが、<br>ときどき反対方向にも更新される | 一方向に更新される |
| 局所解 | 抜け出せる可能性がある | 抜け出せない |

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/local_optimum.png?raw=true" alt="local_optimum" height="200px">





---
## まとめ

　パラメータの数が膨大な場合やコスト関数の形状が複雑な場合、最小二乗法のような"解析的"な手法ではパラメータの最適値（厳密解）を見つけられなくなります。そのようなときに、**勾配法**のような"数値的"にパラメータ推定をおこなう方法が役立ちます。

　今回、勾配法のうち、**確率的勾配降下法**の概要と実装方法を学びました。 確率的勾配降下法は、ランダムに選んだ1サンプルのデータを使ってパラメータを更新していく方法です。

　確率的勾配降下法にはいくつか利点がありますが、とくに重要なのは、**局所解にたどり着いた場合でもそこから抜け出せる可能性がある**点です。これは、パラメータ更新の方向が固定されていない（正の方向に移動することもあれば、負の方向に移動することもある）ためです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2021/images/lr_diff.png?raw=true" alt="gradient_method" height="300px">

[参考:Exploring Stochastic Gradient Descent with Restarts](https://medium.com/38th-street-studios/exploring-stochastic-gradient-descent-with-restarts-sgdr-fa206c38a74e)

　人生も、局所解に陥らないために、時々ふだんとは別の方向に歩んでみるのも良いかもしれません（どこに大域解があるかわかりませんが・・・）。



